# Particle Swarm Optimiser

In [11]:
import os, sys, json 
import numpy as np
sys.path.append(os.path.abspath(".."))

from py_modules.velopix_wrapper.parameter_optimisers import optimiserBase
from py_modules.velopix_wrapper.velopix_pipeline import Pipeline_TrackFollowing
from py_modules.velopix_wrapper.ReconstructionAlgorithms import ReconstructionAlgorithms

## Implement the optimiser child class

In [12]:
from copy import deepcopy
import random 

class ParticleSwarm(optimiserBase):
    def __init__(self,
        swarm_size: int = 10,
        w: float = 0.5,
        c1: float = 1.5,
        c2: float = 1.5,
        convergence_tolerance: float = 1e-4,
        patience: int = 15,
        **kwargs):
        super().__init__()
        self.swarm_size = swarm_size
        self.w = w
        self.c1 = c1
        self.c2 = c2
        self.convergence_tolerance = convergence_tolerance
        self.patience = patience

        # These attributes will be set during init()
        self.swarm = []         # List of particle positions (each a dict)
        self.velocities = []    # List of particle velocities (dicts matching the particle structure)
        self.pbest = []         # Personal best positions for each particle
        self.pbest_scores = []  # Best scores for each particle
        self.iterations = 0
        self.current_particle_index = 0  # Which particle's turn to be updated/evaluated
        self.score_history = []

    def is_finished(self):
        if len(self.score_history) > self.patience:
            recent_scores = self.score_history[-self.patience:]
            improvement = max(recent_scores) - min(recent_scores)
            if improvement < self.convergence_tolerance:
                return True
        return False
    
    def init(self):
        schema = self._algorithm.get_config() # get the schema for the track reconstruction algo 

        self.swarm = []
        self.velocities = []
        self.pbest = []
        self.pbest_scores = []
        self.score_history = []

        for _ in range(self.swarm_size):
            particle = {}
            velocity = {}
            for key, (expected_type, _) in schema.items():
                # init bools at random
                if expected_type == bool:
                    particle[key] = random.choice([True, False])
                    velocity[key] = 0.0
        
                low, high = self._algorithm._bounds().get(key)
                particle[key] = expected_type(random.uniform(low, high))
                velocity[key] = 0.0

            self.swarm.append(particle)
            self.velocities.append(velocity)
            self.pbest.append(deepcopy(particle))
            self.pbest_scores.append(float("inf"))

        self.best_score = float("inf")
        self.best_config = deepcopy(self.swarm[0])
        self.iterations = 0
        self.current_particle_index = 0
        return deepcopy(self.swarm[0])
    
    def next(self):
        # Cycle to the next particle in the swarm, since we need to eval each particle at the time in the algo's
        self.current_particle_index = (self.current_particle_index + 1) % self.swarm_size
        idx = self.current_particle_index
        schema = self._algorithm.get_config()

        for key, (expected_type, _) in schema.items():
            if expected_type == bool:
                # note sure what to do here yet
                continue 

            low, high = self._algorithm._bounds().get(key)
            r1 = random.random()
            r2 = random.random()
            current_vel = self.velocities[idx][key]
            current_pos = self.swarm[idx][key]
            pbest_pos = self.pbest[idx][key]
            gbest_contrib = 0.0 if self.best_score == float("inf") else self.best_config[key] - current_pos

            new_vel = (
                self.w * current_vel +
                self.c1 * r1 * (pbest_pos - current_pos) +
                self.c2 * r2 * gbest_contrib
            )
            self.velocities[idx][key] = new_vel

            new_pos = current_pos + new_vel
            new_pos = max(low, min(new_pos, high))
            self.swarm[idx][key] = expected_type(new_pos)

        self.iterations += 1
        candidate = deepcopy(self.swarm[idx])

        candidate_score = self.score()
        idx = self.current_particle_index
        self.score_history.append(candidate_score)

        if candidate_score < self.pbest_scores[idx]:
            self.pbest_scores[idx] = candidate_score
            self.pbest[idx] = deepcopy(self.swarm[idx])

        if candidate_score < self.best_score:
            self.best_score = candidate_score
            self.best_config = deepcopy(self.swarm[idx])

        return candidate
    
    def score(self): # this is a very basic optimalisation func, needs to be improved for certain
        result = self.get_run_data()

        n_tracks = result.get("total_tracks")
        if n_tracks <= 0:
            return float("inf") # since the optimiser found a way to get 0 tracks and thus a bizar low score we penalise any set of parameters that cause this

        ghost_rate = result.get("overall_ghost_rate") # minimise
        clones = result.get("categories") 
        clone_pct = 0
        for clone in clones:
            clone_pct += clone.get("clone_percentage")
        return (ghost_rate + clone_pct / len(clones)) * 1000 / n_tracks

**Load event data**

In [13]:
events = []
n_files = 100

for i in range(0, n_files):
    if i == 51:
        """
        There's an issue with event 51 -> module_prefix_sum contains value 79 twice resulting in and indexing error when loading the event
        """
        print(f"Skipping problematic file: velo_event_{i}.json")
    else:    
        print(f"Loading file: velo_event_{i}.json")
        event_file = open(os.path.join("../DB/raw", f"velo_event_{i}.json"))
        json_data = json.loads(event_file.read())
        events.append(json_data)
        event_file.close()

Loading file: velo_event_0.json
Loading file: velo_event_1.json
Loading file: velo_event_2.json
Loading file: velo_event_3.json
Loading file: velo_event_4.json
Loading file: velo_event_5.json
Loading file: velo_event_6.json
Loading file: velo_event_7.json
Loading file: velo_event_8.json
Loading file: velo_event_9.json
Loading file: velo_event_10.json
Loading file: velo_event_11.json
Loading file: velo_event_12.json
Loading file: velo_event_13.json
Loading file: velo_event_14.json
Loading file: velo_event_15.json
Loading file: velo_event_16.json
Loading file: velo_event_17.json
Loading file: velo_event_18.json
Loading file: velo_event_19.json
Loading file: velo_event_20.json
Loading file: velo_event_21.json
Loading file: velo_event_22.json
Loading file: velo_event_23.json
Loading file: velo_event_24.json
Loading file: velo_event_25.json
Loading file: velo_event_26.json
Loading file: velo_event_27.json
Loading file: velo_event_28.json
Loading file: velo_event_29.json
Loading file: velo_e

In [14]:
pipeline = Pipeline_TrackFollowing(events=events, intra_node=False)

In [15]:
Optimiser = ParticleSwarm() # Lets use default values for now 
optimal_parameters = pipeline.optimise_parameters(Optimiser, max_runs=1000) # DO NOT remove max_runs, chances are that this will run forever

In [16]:
print(optimal_parameters) # Note these are just here for example...

{'x_slope': 23.61003906122083, 'y_slope': 51.2491011482025, 'x_tol': 2.6326128439547194, 'y_tol': 3.7118928972189504, 'scatter': 2.7948810889363034}


In [17]:
print(Optimiser.best_score)

0.777965858122696


In [18]:
print(Optimiser.is_finished())

False


In [19]:
print(Optimiser.score_history)

[1.1780858797885796, 1.4396309641266976, 1.5814234050809048, 1.4458784315890756, 1.4724989951571235, 1.5307153639129194, 1.5350733945592077, 1.425335465681102, 1.5424402161126152, 1.4901264388987925, 1.2355719425667633, 1.4396309641266976, 1.4230263930554263, 1.5656926666134936, 1.5959985838030284, 1.4139449579872359, 1.5471849643691808, 1.521930002112272, inf, 1.325708090631762, 1.58019717287882, 1.4396309641266976, 1.5804694700922268, 1.5750814827365636, 1.5139107437845374, 1.5113361968583352, 1.4284165459978293, 1.428473674124358, 1.3838663209920354, 1.4052845323456202, inf, 1.4396309641266976, 1.5714601578197192, 1.5467703377044704, 1.4352479063437003, 1.5530307660411284, 1.5711597513342657, 1.4019237438289105, 1.442826056734382, 1.2808730898891623, 1.5001175167904235, 1.4396309641266976, 1.433268836269622, 1.5684131570466278, 1.4751521921156348, 1.5168541909393254, 1.5710865375239806, 1.451888956454339, 1.4323023673287485, 1.4286196221918968, 1.3636943474993923, 1.4396309641266976

In [20]:
result = Optimiser.get_run_data()
print(result)

{'total_tracks': 71866, 'total_ghosts': 69750, 'overall_ghost_rate': 97.05563131383408, 'event_avg_ghost_rate': 94.99073438738132, 'categories': [{'label': 'long_fromb', 'n_reco': 31, 'n_particles': 444, 'recoeffT': 6.981981981981982, 'avg_recoeff': 6.981981981981981, 'n_clones': 2, 'clone_percentage': 6.451612903225806, 'purityT': 78.91219495497569, 'avg_purity': 0.0, 'avg_hiteff': 0.0, 'hit_eff_percentage': 63.077920791824525}, {'label': 'long_strange>5GeV', 'n_reco': 4, 'n_particles': 174, 'recoeffT': 2.2988505747126435, 'avg_recoeff': 2.2988505747126435, 'n_clones': 0, 'clone_percentage': 0.0, 'purityT': 90.0, 'avg_purity': 0.0, 'avg_hiteff': 0.0, 'hit_eff_percentage': 78.33333333333333}, {'label': 'long_fromb>5GeV', 'n_reco': 23, 'n_particles': 343, 'recoeffT': 6.705539358600583, 'avg_recoeff': 6.705539358600583, 'n_clones': 0, 'clone_percentage': 0.0, 'purityT': 80.12641947424555, 'avg_purity': 0.0, 'avg_hiteff': 0.0, 'hit_eff_percentage': 63.40191209756427}, {'label': 'velo', 'n